Source: https://learn.microsoft.com/en-us/training/modules/ai-foundry-sdk/

In [4]:
# package installation:
# pip install azure-ai-textanalytics azure-ai-projects azure-identity azure-ai-inference python-dotenv

In [5]:
## load in environment variables from .env file
import os
from dotenv import load_dotenv

load_dotenv()
PROJECT_CONNECTION = os.getenv("PROJECT_CONNECTION") # project connection string from overview page in Azure AI Foundry
MODEL_DEPLOYMENT =  os.getenv("MODEL_DEPLOYMENT")    # model name (gpt-4)
API_VERSION = os.getenv("API_VERSION")               # model version (2024-12-01-preview)

## 1. Connect to Azure AI Project

The core package for working with projects in the Azure AI Foundary SDK is the `Azure AI Projects` library (**AIProjectClient** class) which serves as a programmatic proxy for a project enabling us to connect to an Azure AI Foundary project and access connected resources defined within it.

The package `azure-identity` is to allow for a keyless authentication using the default Azure credentials (before so need to install Azure CLI and sign into azure before running the code to set up an authenticated Azure session - see below).

**Prerequisites**: 
1. get the **project connection string** from the Azure AI Foundry overview page.
2. Install **Azure CLI** per the [installation instructions](https://learn.microsoft.com/en-gb/cli/azure/install-azure-cli?WT.mc_id=academic-105485-koreyst) to enable authentication. Once installed, open a terminal and run `az login` to sign in to Azure account in vscode.


In [6]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential

project_connection_string = PROJECT_CONNECTION
project_client = AIProjectClient.from_connection_string(
      credential=DefaultAzureCredential(),
      conn_str=project_connection_string,
    )

Each Azure AI Foundry project includes **connected resources** which can be defined both at hub and project level. Each resource is connected to an external service such as Azure AI services, Azure AI search and others. To retrieve connections in python, the **AIProjectClient** object in python has a **connections** property:

In [7]:
# access a full collection of connection objects
project_client.connections.list()

In [8]:
# get default connection of specified type (Azure AI Service)
project_client.connections.get_default(connection_type=ConnectionType.AZURE_AI_SERVICES)

## 2. Create a TextAnalyticsClient object for sentiment prediction

In [9]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

try:
    # Get project client
    project_connection_string = PROJECT_CONNECTION
    project_client = AIProjectClient.from_connection_string(
      credential=DefaultAzureCredential(),
      conn_str=project_connection_string,
    )

    # Get the properties of the default Azure AI Services connection with credentials
    connection = project_client.connections.get_default(
      connection_type=ConnectionType.AZURE_AI_SERVICES,
      include_credentials=True, 
    )

    # Use the connection information to create a text analytics client
    ai_svc_credential = AzureKeyCredential(connection.key)
    text_analytics_client = TextAnalyticsClient(endpoint=connection.endpoint_url, credential=ai_svc_credential)

    # Use the Language service to analyze some text (to infer sentiment) 
    text = "I hated the movie. It was so slow!"
    sentimentAnalysis = text_analytics_client.analyze_sentiment(documents=[text])[0]
    print("Text: {}\nSentiment: {}".format(text,sentimentAnalysis.sentiment))

except Exception as ex:
    print(ex)

Text: I hated the movie. It was so slow!
Sentiment: negative


## 3. Create a chat client

### Prerequisite: Deploy a gpt-4 model in Azure AI Foundry Studio
**1. Create a project at Azure AI Foundry portal at https://ai.azure.com:**
- project name: `my-ai-project`
- hub name: `my-ai-hub`
- subscription: your azure subscription
- resource group: a unique resource name (e.g. `my-ai-resources`)
- Location: select `gpt-4` and use the recommended region.
- connect Azure AI service or Azure OpenAI: create a new AI Services resource (`my-ai-resources`)
- Connect Azure AI Search: skip connecting

**2. Deploy a Gen AI Model**
- In the toolbar at the top right of your Azure AI Foundry project page, use the **Preview features** icon to enable the **Deploy models to Azure AI model inference service** feature. This feature ensures your model deployment is available to the Azure AI Inference service, which you’ll use in your application code.
- **My assets** > select **Models + endpoints** > select **Deploy model** dropdown > **Deploy base model** > search for **gpt-4**
- deploy the model with the following settings:
    - Deployment name: gpt-4
    - Deployment type: select default version
    - Model version: default
    - Connected AI resource: your azure OpenAI resource connection
    - Tokens per Minute Rate Limit (thousands): maximum available
    - Content filter: DefaultV2
    - Enable dynamic quota: disabled

**3. Set up application configs in .env file**
- defining project connection string and deployment model name

In [11]:
## EXAMPLE 1: SIMPLE SINGLE QUERY

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

try:
    # Initialize the project client
    project_connection_string = PROJECT_CONNECTION
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=project_connection_string,
    )

    ## Get an Azure OpenAI chat client
    openai_client = project_client.inference.get_azure_openai_client(api_version=API_VERSION)

    # Get a chat completion based on a user-provided prompt
    user_prompt = input("Enter a question:")
    print('Question:', user_prompt)
    response = openai_client.chat.completions.create(
        model=MODEL_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that answers questions."},
            {"role": "user", "content": user_prompt},
        ]
    )
    print(response.choices[0].message.content)

except Exception as ex:
    print(ex)

Question: What is the shape of earth?
The shape of Earth is best described as an oblate spheroid. This means that it is mostly spherical, but slightly flattened at the poles and bulging at the equator due to its rotation.


In [13]:
## EXAMPLE 2: MULTIPLE QUERY WITH CHAT HISTORY (see file chat-app.py in the same dir)

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage

def main(): 

    # Clear the console
    # os.system('cls' if os.name=='nt' else 'clear')
        
    try: 
    
        # Initialize the project client
        projectClient = AIProjectClient.from_connection_string(
            conn_str=PROJECT_CONNECTION,
            credential=DefaultAzureCredential()
        )

        ## Get a chat client
        chat = projectClient.inference.get_chat_completions_client()

        ## Initialize prompt with system message
        prompt=[
         SystemMessage("You are a helpful AI assistant that answers questions.")
        ]

        # Loop until the user types 'quit'
        while True:
            # Get input text
            input_text = input("Enter the prompt (or type 'quit' to exit): ")
            if input_text.lower() == "quit":
                break
            if len(input_text) == 0:
                print("Please enter a prompt.")
                continue
            print('Question:', input_text)
            
            # Get a chat completion
            prompt.append(UserMessage(input_text))
            response = chat.complete(
                model=MODEL_DEPLOYMENT,
                messages=prompt)
            completion = response.choices[0].message.content
            print(completion)
            prompt.append(AssistantMessage(completion))


    except Exception as ex:
        print(ex)

if __name__ == '__main__': 
    main()


Question: Hello who are you?
Hello! I'm an AI assistant here to help answer your questions and provide information on a wide range of topics. How can I assist you today?
Question: What is the slowest walking animal on Earth?
The slowest walking animal on Earth is the three-toed sloth. It moves at a leisurely pace of about 0.03 miles per hour on the ground. These creatures are much better adapted to life in the trees of tropical rainforests, where they spend most of their time hanging from branches and moving through the canopy.
